# Apache Cassandra Keyspace



In [1]:
import numpy as np
import pandas as pd
import cassandra
import re
import os
import glob
import json
import csv

from cassandra.cluster import Cluster

def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

In [2]:
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root, '*'))

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 


csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName',
                     'length', 'level','location','sessionId','song','userId'])

    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6],
                         row[7], row[8], row[12], row[13], row[16]))

In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [6]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS song_trendy 
        WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)

except Exception as e:
    print(e)


session.set_keyspace('song_trendy')

<br>

### Query 1

`artist`, `song` and `length` for `sessionId=338` and `itemInSession=4`

In [7]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS song_by_item (
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            length FLOAT,
            PRIMARY KEY (sessionId, itemInSession));
    """)
except Exception as e:
    print(e)

In [8]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO song_by_item (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [9]:
session.row_factory = pandas_factory

result = session.execute("""
    SELECT artist, song, length
    FROM song_by_item
    WHERE
        sessionId=338
""")

df = result._current_rows 
df

,artist,song,length
0,Pixies,Build High,89.364441
1,The Roots / Jack Davey,Atonement,155.950577
2,Mike And The Mechanics,A Beggar On A Beach Of Gold,275.121185
3,Faithless,Music Matters (Mark Knight Dub),495.307312


<br>

### Query 2

`artist`, `song` (sorted by `itemInSession`) and user (`firstName` and `lastName`) for `userid=10` and `sessionid=182`

In [10]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS songinfo_by_session (
            userId INT,
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            firstName TEXT,
            lastName TEXT,
            PRIMARY KEY ((userId, sessionId), itemInSession));
    """)
except Exception as e:
    print(e)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO songinfo_by_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [12]:
session.row_factory = pandas_factory

result = session.execute("""
    SELECT artist, song, firstName, lastName, userId
    FROM songinfo_by_session
    WHERE
        userId=10 AND
        sessionId=182
    ORDER BY itemInSession;
""")

df = result._current_rows 
df

,artist,song,firstname,lastname,userid
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz,10
1,Three Drives,Greece 2000,Sylvie,Cruz,10
2,Sebastien Tellier,Kilometer,Sylvie,Cruz,10
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz,10


<br>

### Query 3

all users (`firstName` and `lastName`) who listened to the `song` 'All Hands Against His Own'

In [13]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS userinfo_by_song (
            song TEXT,
            firstName TEXT,
            lastName TEXT,
            userId INT,
            PRIMARY KEY (song, firstName, lastName, userId));
    """)
except Exception as e:
    print(e)

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO userinfo_by_song (song, firstName, lastName, userId)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [28]:
session.row_factory = pandas_factory

result = session.execute("""
    SELECT firstName, lastName, song
    FROM userinfo_by_song
    WHERE song='All Hands Against His Own';
""")

df = result._current_rows 
df

,firstname,lastname,song
0,Jacqueline,Lynch,All Hands Against His Own
1,Sara,Johnson,All Hands Against His Own
2,Tegan,Levine,All Hands Against His Own


<br>

Xoa bang.

In [29]:
try:
    session.execute("DROP TABLE song_by_item")
    session.execute("DROP TABLE songinfo_by_session")
    session.execute("DROP TABLE userinfo_by_song")
except Exception as e:
    print(e)

<br>

Dong ket noi.

In [21]:
session.shutdown()
cluster.shutdown()